# Modelos de Predicción con Gradiente en el Mercado de Coches

## Introducción

### Rusty Bargain es una tienda de coches de segunda mano que está desarrollando una aplicación para ayudar a los usuarios a conocer el valor de mercado de sus vehículos de manera rápida y sencilla. Para lograrlo, es necesario crear un modelo que pueda predecir este valor con precisión.

### En este análisis, se han probado diferentes modelos de aprendizaje automático, como regresión lineal, árboles de decisión, bosque aleatorio y modelos basados en gradiente como XGBoost y LightGBM. El objetivo es encontrar el modelo que ofrezca las mejores predicciones, así como un tiempo de entrenamiento y predicción adecuado para la aplicación móvil.

### Este estudio ayudará a Rusty Bargain a elegir el modelo más eficiente para mejorar la experiencia del usuario en su plataforma.

In [107]:
# Importamos las librerias necesarias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
import catboost
from sklearn.metrics import mean_squared_error, r2_score
from time import time

In [108]:
# Cargamos el dataset

df = pd.read_csv('car_data.csv')	

In [109]:
# Mostramos la informacion del dataset

print(df.info())
print()
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

Algunas columnas tienen valores faltantes, como VehicleType, Gearbox, Model, FuelType, NotRepaired.

In [110]:
# Rellenar los valores faltantes

df['VehicleType'] = df['VehicleType'].fillna(df['VehicleType'].mode()[0])
df['Gearbox'] = df['Gearbox'].fillna(df['Gearbox'].mode()[0])
df['Model'] = df['Model'].fillna(df['Model'].mode()[0])
df['FuelType'] = df['FuelType'].fillna(df['FuelType'].mode()[0])
df['NotRepaired'] = df['NotRepaired'].fillna(df['NotRepaired'].mode()[0])


print(df.head())


        DateCrawled  Price VehicleType  RegistrationYear Gearbox  Power  \
0  24/03/2016 11:52    480       sedan              1993  manual      0   
1  24/03/2016 10:58  18300       coupe              2011  manual    190   
2  14/03/2016 12:52   9800         suv              2004    auto    163   
3  17/03/2016 16:54   1500       small              2001  manual     75   
4  31/03/2016 17:25   3600       small              2008  manual     69   

   Model  Mileage  RegistrationMonth  FuelType       Brand NotRepaired  \
0   golf   150000                  0    petrol  volkswagen          no   
1   golf   125000                  5  gasoline        audi         yes   
2  grand   125000                  8  gasoline        jeep          no   
3   golf   150000                  6    petrol  volkswagen          no   
4  fabia    90000                  7  gasoline       skoda          no   

        DateCreated  NumberOfPictures  PostalCode          LastSeen  
0  24/03/2016 00:00               

In [111]:
# Eliminamos datos irrelevantes para el aprendizaje

df = df.drop(['PostalCode', 'NumberOfPictures', 'DateCrawled', 'DateCreated', 'LastSeen'], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Price              354369 non-null  int64 
 1   VehicleType        354369 non-null  object
 2   RegistrationYear   354369 non-null  int64 
 3   Gearbox            354369 non-null  object
 4   Power              354369 non-null  int64 
 5   Model              354369 non-null  object
 6   Mileage            354369 non-null  int64 
 7   RegistrationMonth  354369 non-null  int64 
 8   FuelType           354369 non-null  object
 9   Brand              354369 non-null  object
 10  NotRepaired        354369 non-null  object
dtypes: int64(5), object(6)
memory usage: 29.7+ MB
None


In [112]:
# Aplicamos One Hot Encoding a todas las columnas categóricas

df = pd.get_dummies(df, columns=['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first=True)


In [113]:
print(df.head())

   Price  RegistrationYear  Power  Mileage  RegistrationMonth  \
0    480              1993      0   150000                  0   
1  18300              2011    190   125000                  5   
2   9800              2004    163   125000                  8   
3   1500              2001     75   150000                  6   
4   3600              2008     69    90000                  7   

   VehicleType_convertible  VehicleType_coupe  VehicleType_other  \
0                    False              False              False   
1                    False               True              False   
2                    False              False              False   
3                    False              False              False   
4                    False              False              False   

   VehicleType_sedan  VehicleType_small  ...  Brand_skoda  Brand_smart  \
0               True              False  ...        False        False   
1              False              False  ...        

Ahora el DataFrame está bien estructurado para continuar con el modelo de Machine Learning, con las variables categóricas convertidad en columnas y las variables continuas permanecen en su formato original, además las fechas se mantienen como datetime.

In [114]:
# Dividimos los datos en entrenamiento y prueba

X = df.drop('Price', axis=1)
y = df['Price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1234)

In [119]:
# Definimos el modelo de regresión lineal

model = LinearRegression()
start_time = time()
model.fit(X_train, y_train)
train_time = time() - start_time

print(f'Tiempo de entremamiento = {train_time:.2f} segundos')

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f'r2_score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

Tiempo de entremamiento = 3.60 segundos
r2_score: 0.49
RMSE: 3219.70


El modelo tuvo un tiempo de entranamiento de 3.60 segundos y un RMSE de 3219.70 indica que en promedio, las predicciones del modelo se desvían en alrededor de 3219 unidades de los valores reales.

In [120]:
# Definimos el modelo de árbol de decisión

model = DecisionTreeRegressor(max_depth=15, random_state=1234)
start_time = time()
model.fit(X_train, y_train)
train_time = time() - start_time
print(f'Tiempo de entremamiento = {train_time:.2f} segundos')
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'r2_score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

Tiempo de entremamiento = 6.29 segundos
r2_score: 0.79
RMSE: 2047.19


El modelo de arbol de decisión tuvo un tiempo de entrenamiento de 6.25 segundos y un RMSE de 2047.19 lo cual fue un poco más bajo que en el modelo de regresión lineal.

In [121]:
# Definamos el modelo de bosque aleatorio

model = RandomForestRegressor(n_estimators=50, max_depth=15, random_state=1234)
start_time = time()
model.fit(X_train, y_train)
train_time = time() - start_time
print(f'Tiempo de entremamiento = {train_time:.2f} segundos')
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'r2_score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')


Tiempo de entremamiento = 198.01 segundos
r2_score: 0.83
RMSE: 1848.19


El modelo de bosque aleatorio tuvo un tiempo de entrenamiento de 198.01 segundos, siendo el que más se ha tardado y con bastante diferencia, pero tuvo el mejor RMSE con 1848.19.

In [122]:
# Definamos el modelo de LightGBM

model_lgb = lgb.LGBMRegressor(n_estimators=50, max_depth=15, verbose=2, random_state=1234)
start_time = time()
model_lgb.fit(X_train, y_train)
train_time = time() - start_time
print()
print(f'Tiempo de entremamiento = {train_time:.2f} segundos')
y_pred = model_lgb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'r2_score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.911444
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.554301
[LightGBM] [Debug] init for col-wise cost 0.005444 seconds, init for row-wise cost 0.016076 seconds
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008389 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 953
[LightGBM] [Info] Number of data points in the train set: 265776, number of used features: 291
[LightGBM] [Info] Start training from score 4411.233366
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 6
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 7
[LightGBM] [Debug] Trained 

El tiempo de entrenamiento para el modelo de LightGBM fue de 0.96 segundos, lo cual es muy rápido considerando el tamaño del conjunto de datos y el RMSE fue de 1939.59, este valor no es el más bajo de todos los modelos pero si el que más rápido lo hace.

In [123]:
# Definamos el modelo de XGBoost

model_xgb = XGBRegressor(n_estimators=50, max_depth=15, learning_rate=0.1, verbosity=2, random_state=1234)
start_time = time()
model_xgb.fit(X_train, y_train)
train_time = time() - start_time
print(f'Tiempo de entremamiento = {train_time:.2f} segundos')
y_pred = model_xgb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'r2_score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')

Tiempo de entremamiento = 6.79 segundos
r2_score: 0.85
RMSE: 1729.12


El modelo de XGBoost está funcionando bastante bien, con un r2 de 0.85 y un RMSE de 1729.12, está logrando un buen balance entre precisión y desempeño, aunque el tiempo de entrenamiento no es lo más rápido comparado con otros modelos, aún siue siendo un tiempo razonable.

In [124]:
# Definamos el modelo de CatBoost

model_cat = catboost.CatBoostRegressor(n_estimators=50, max_depth=15, learning_rate=0.1, verbose=0, random_state=1234)
start_time = time()
model_cat.fit(X_train, y_train)
train_time = time() - start_time
print(f'Tiempo de entremamiento = {train_time:.2f} segundos')
y_pred = model_cat.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'R2 Score: {r2:.2f}')
print(f'RMSE: {rmse:.2f}')


Tiempo de entremamiento = 19.24 segundos
R2 Score: 0.83
RMSE: 1870.06


El modelo de CatBoost también está mostrando buenos resultados, aunque con un R2 de 0.83 y un RMSE de 1870.08 es ligeramente inferior al desempeño de XGBoost, sin embargo su tiempo de entrenamiento es más alto comparado con otros modelos.

## Conclusión

En el análisis de los modelos de predicción del valor de mercado de coches para Rusty Bargain, se probaron diferentes algoritmos:
1. XGBoost resultó ser el mejor en cuanto a RMSE (1729.12) y R2 (0.85). Este modelo presentó la mayor precisión en las predicciones y una excelente capacidad de ajuste a los datos. Además, su tiempo de entrenamiento (6.79 segundos) es razonable.

2. CatBoost también mostró buenos resultados con un R2 de 0.83 y un RMSE de 1870.06, siendo ligeramente menos preciso que XGBoost, pero aún así ofreciendo un buen desempeño.

3. LightGBM presentó un RMSE de 1939.59, lo que lo hace competitivo, aunque con menor precisión en comparación con XGBoost.

4. Regresión Lineal tuvo un RMSE de 3219.70, el valor más alto entre todos los modelos. Esto indica que la regresión lineal tiene una menor capacidad para ajustar bien los datos en comparación con los modelos basados en árboles de decisión y potenciación del gradiente.

En resumen, XGBoost es la mejor opción para Rusty Bargain en términos de precisión y velocidad de predicción, mientras que la regresión lineal y el random forest no es tan efectiva debido a su menor capacidad de ajuste a los datos y tiempo de ejecución.